In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

from sklearn.linear_model import LogisticRegression


import matplotlib.pyplot as plt

In [2]:
gene_effect_df = pd.read_csv("GeneMutation.csv").rename(columns = {'Unnamed: 0': 'ModelID'})
model_df = pd.read_csv("Model.csv")

merged_df = pd.merge(gene_effect_df, model_df[['ModelID', 'OncotreePrimaryDisease']], on='ModelID', how='inner')

new_cols = gene_effect_df.columns.tolist()
new_cols.insert(1, "OncotreePrimaryDisease")

gene_df = merged_df[new_cols]

C:\Users\adenl\AppData\Local\Temp\ipykernel_15236\3501280885.py:1: DtypeWarning: Columns (20,48,52,54,55,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_effect_df = pd.read_csv("GeneMutation.csv").rename(columns = {'Unnamed: 0': 'ModelID'})


In [3]:
categorical_mapping = {item: idx for idx, item in enumerate(list(gene_df['OncotreePrimaryDisease'].unique()))}
gene_df['Disease as Number'] = gene_df['OncotreePrimaryDisease'].map(categorical_mapping)

gene_df = gene_df.fillna(0)

gene_df = gene_df.set_index('ModelID')

# gene_df.head()

C:\Users\adenl\AppData\Local\Temp\ipykernel_15236\369468172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['Disease as Number'] = gene_df['OncotreePrimaryDisease'].map(categorical_mapping)


In [4]:
X = gene_df.drop(columns = ['Disease as Number', 'OncotreePrimaryDisease'])
y = gene_df['Disease as Number']

In [6]:
categorical_cols = X.select_dtypes(include=['object']).columns
X = X.drop(columns = categorical_cols)

In [7]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y, test_size=0.1)

model_log = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', max_iter = 1000)
model_log.fit(X_train_log, y_train_log)

y_pred_log = model_log.predict(X_test_log)

accuracy_log = accuracy_score(y_test_log, y_pred_log)
print(accuracy_log)

0.16511778041347006


C:\Users\adenl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# Couldn't complete due to long run time
# kf = KFold(n_splits = 5, random_state = 1, shuffle=True)

# cv_scores_log = cross_val_score(model_log, X, y, cv=kf)
# avg_score_log = np.mean(cv_scores_log)
# print(cv_scores_log)
# print(avg_score_log)

In [10]:
reverse_map = {idx: item for idx, item in enumerate(list(gene_df['OncotreePrimaryDisease'].unique()))}

coef_map = {}

coef = list(model_log.coef_)

for i in range(84):
    # Get indices of top 5 coefficients
    top_5_indices = sorted(range(len(coef[i])), key=lambda x: abs(coef[i][x]), reverse=True)[:5]
    # Map top 5 coefficients to their corresponding features
    top_5_features = [X.columns[index] for index in top_5_indices]
    # Store in coef_map
    coef_map[reverse_map[i]] = top_5_features

for category, top_5_features in coef_map.items():
    print(f"Top 5 features for {category}: {top_5_features}")

Top 5 features for Non-Small Cell Lung Cancer: ['Pos', 'EntrezGeneID', 'GwasPmID', 'PS', 'DP']
Top 5 features for Esophageal Squamous Cell Carcinoma: ['Pos', 'EntrezGeneID', 'GwasPmID', 'PS', 'DP']
Top 5 features for Invasive Breast Carcinoma: ['Pos', 'GwasPmID', 'EntrezGeneID', 'PS', 'DP']
Top 5 features for Head and Neck Squamous Cell Carcinoma: ['Pos', 'GwasPmID', 'EntrezGeneID', 'PS', 'DP']
Top 5 features for Prostate Adenocarcinoma: ['Pos', 'GwasPmID', 'PS', 'EntrezGeneID', 'DP']
Top 5 features for Embryonal Tumor: ['Pos', 'GwasPmID', 'PS', 'EntrezGeneID', 'DP']
Top 5 features for Ovarian Epithelial Tumor: ['Pos', 'GwasPmID', 'EntrezGeneID', 'PS', 'DP']
Top 5 features for Esophagogastric Adenocarcinoma: ['Pos', 'GwasPmID', 'EntrezGeneID', 'PS', 'RefCount']
Top 5 features for Hepatocellular Carcinoma: ['Pos', 'GwasPmID', 'EntrezGeneID', 'PS', 'DP']
Top 5 features for Endometrial Carcinoma: ['Pos', 'EntrezGeneID', 'PS', 'GwasPmID', 'DP']
Top 5 features for Colorectal Adenocarcinoma:

In [11]:
df_coef = pd.DataFrame(coef_map)

df_coef = df_coef.transpose()

df_coef.fillna('', inplace=True)

df_coef.reset_index(inplace=True)

df_coef.rename(columns={'index': 'Primary Disease'}, inplace=True)

new_columns = {col: f"#{i+1} Most Weighted Gene" for i, col in enumerate(df_coef.columns[1:])}
df_coef.rename(columns=new_columns, inplace=True)

df_coef

,Primary Disease,#1 Most Weighted Gene,#2 Most Weighted Gene,#3 Most Weighted Gene,#4 Most Weighted Gene,#5 Most Weighted Gene
0,Non-Small Cell Lung Cancer,Pos,EntrezGeneID,GwasPmID,PS,DP
1,Esophageal Squamous Cell Carcinoma,Pos,EntrezGeneID,GwasPmID,PS,DP
2,Invasive Breast Carcinoma,Pos,GwasPmID,EntrezGeneID,PS,DP
3,Head and Neck Squamous Cell Carcinoma,Pos,GwasPmID,EntrezGeneID,PS,DP
4,Prostate Adenocarcinoma,Pos,GwasPmID,PS,EntrezGeneID,DP
...,...,...,...,...,...,...
79,Myelodysplastic Syndromes,Pos,PS,EntrezGeneID,GwasPmID,DP
80,Prostate Small Cell Carcinoma,Pos,EntrezGeneID,PS,GwasPmID,DP
81,Ovarian Germ Cell Tumor,Pos,PS,GwasPmID,EntrezGeneID,DP
82,Epithelioid Sarcoma,EntrezGeneID,Pos,PS,GwasPmID,DP
